# Funciones de Preprocesamiento y Analisis de Data set

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

# Configuramos estilo de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

In [ ]:
def analizar_dataframe(df):
    """
    Función que recibe un dataframe y muestra información sobre su estructura,
    valores nulos y duplicados.
    
    Parámetros:
    df (DataFrame): El dataframe a analizar
    """
    print(f"Número de filas y columnas: {df.shape}\n")
    print(f"Número de nulos por columnas: \n{df.isnull().sum()}\n")
    print(f"Número total de nulos: {df.isna().sum().values.sum()} \n")
    print(f"% de nulos de cada columna: \n{df.isna().mean()*100} \n")
    print("Porcentaje de duplicados:",
          f"{100 * (len(df) - len(df.drop_duplicates())) / len(df):.2f}%")

In [ ]:
# Función para crear todas las visualizaciones de distribución
def plot_distributions(df, figsize=(18, 24)):
    """
    Crea gráficos de distribución (histograma y KDE) para todas las variables numéricas.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        figsize (tuple): Tamaño de la figura.
    """
    # Determinamos el número de variables y configuramos los subplots
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    n_vars = len(numerical_cols)
    rows = int(np.ceil(n_vars / 2))
    
    # Creamos la figura y los subplots
    fig, axes = plt.subplots(rows, 2, figsize=figsize)
    axes = axes.flatten()
    
    # Iteramos sobre cada variable y creamos sus visualizaciones
    for i, col in enumerate(numerical_cols):
        # Histograma con KDE
        sns.histplot(df[col], kde=True, ax=axes[i], color='royalblue')
        
        # Añadimos líneas verticales para estadísticas clave
        median = df[col].median()
        mean = df[col].mean()
        
        axes[i].axvline(median, color='red', linestyle='--', linewidth=1.5, 
                        label=f'Mediana: {median:.2f}')
        axes[i].axvline(mean, color='green', linestyle='-', linewidth=1.5, 
                       label=f'Media: {mean:.2f}')
        
        # Configuramos títulos y etiquetas
        axes[i].set_title(f'Distribución de {col}', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)
        axes[i].set_ylabel('Frecuencia', fontsize=12)
        axes[i].legend(fontsize=10)
        
        # Añadimos texto con estadísticas adicionales
        skew = df[col].skew()
        kurtosis = df[col].kurtosis()
        axes[i].text(0.02, 0.95, f'Asimetría: {skew:.2f}\nCurtosis: {kurtosis:.2f}', 
                    transform=axes[i].transAxes, fontsize=10,
                    bbox=dict(facecolor='white', alpha=0.8))
    
    # Ocultamos los ejes vacíos si el número de variables es impar
    if n_vars % 2 != 0:
        axes[-1].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Histogramas y Distribuciones de las Variables', fontsize=18, y=1.005)
    return fig

In [ ]:
# Función para crear boxplots y detectar outliers
def plot_boxplots(df, figsize=(18, 24)):
    """
    Crea boxplots para todas las variables numéricas e identifica outliers.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        figsize (tuple): Tamaño de la figura.
    """

    # Determinamos el número de variables y configuramos los subplots Select only numerical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    n_vars = len(numerical_cols)
    rows = int(np.ceil(n_vars / 2))
    
    # Creamos la figura y los subplots
    fig, axes = plt.subplots(rows, 2, figsize=figsize)
    axes = axes.flatten()
    
    # Iteramos sobre cada variable y creamos sus boxplots
    for i, col in enumerate(numerical_cols):
        # Calculamos los límites para los outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identificamos los outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        n_outliers = len(outliers)
        
        # Creamos el boxplot
        sns.boxplot(x=df[col], ax=axes[i], color='lightgreen', width=0.5)
        
        # Superponemos el strip plot para ver la distribución de puntos
        sns.stripplot(x=df[col], ax=axes[i], color='darkblue', alpha=0.3, size=4)
        
        # Configuramos títulos y etiquetas
        axes[i].set_title(f'Boxplot de {col} ({n_outliers} outliers)', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)
        
        # Añadimos texto con estadísticas sobre los outliers
        pct_outliers = (n_outliers / len(df)) * 100
        axes[i].text(0.02, 0.95, 
                    f'Outliers: {n_outliers} ({pct_outliers:.1f}%)\n' + 
                    f'Rango normal: [{lower_bound:.2f}, {upper_bound:.2f}]',
                    transform=axes[i].transAxes, fontsize=10,
                    bbox=dict(facecolor='white', alpha=0.8))
    
    # Ocultamos los ejes vacíos si el número de variables es impar
    if n_vars % 2 != 0:
        axes[-1].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Boxplots con Detección de Outliers', fontsize=18, y=1.005)
    return fig

In [ ]:
# Función para crear pairplots para variables seleccionadas
def plot_pairplot(df, target='MEDV', features=None):
    """
    Crea pairplots para las variables más importantes relacionadas.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        target (str): Nombre de la variable objetivo.
        features (list): Lista de características a incluir. Si es None, se seleccionan
                      automáticamente las más correlacionadas con el target.
    """
    if features is None:
        # Seleccionamos las 5 variables más correlacionadas con MEDV
        corr_with_target = df.corr()[target].abs().sort_values(ascending=False)
        features = corr_with_target.index[1:6].tolist()  # Excluimos el target mismo
    
    # Creamos el subset de datos
    plot_data = df[features + [target]].copy()
    
    # Creamos el pairplot
    fig = sns.pairplot(plot_data, height=2.5, 
                      diag_kind='kde', 
                      plot_kws={'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
                      corner=True)
    
    fig.fig.suptitle('Pairplot de Variables Importantes vs MEDV', fontsize=18, y=1.02)
    
    return fig.fig

In [ ]:
# Función para crear la matriz de correlación
def plot_correlation_matrix(df, figsize=(14, 12)):
    """
    Crea una matriz de correlación para todas las variables.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        figsize (tuple): Tamaño de la figura.
    """
    # Calculamos la matriz de correlación valores numéricos
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df = df[numerical_cols]
    # Calculamos la matriz de correlación
    corr_matrix = df.corr()
    
    # Creamos la figura
    fig, ax = plt.subplots(figsize=figsize)
    
    # Creamos el mapa de calor
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .8}, annot=True, fmt=".2f")
    
    # Configuramos título
    plt.title('Matriz de Correlación', fontsize=18)
    
    return fig

In [ ]:
# Función para crear ECDFs (Empirical Cumulative Distribution Function)
def plot_ecdfs(df, figsize=(18, 24)):
    """
    Crea gráficos ECDF para todas las variables numéricas.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        figsize (tuple): Tamaño de la figura.
    """
    # Determinamos el número de variables y configuramos los subplots
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    n_vars = len(numerical_cols)
    rows = int(np.ceil(n_vars / 2))
    
    # Creamos la figura y los subplots
    fig, axes = plt.subplots(rows, 2, figsize=figsize)
    axes = axes.flatten()
    
    # Iteramos sobre cada variable y creamos sus ECDFs
    for i, col in enumerate(numerical_cols):
        # Ordenamos los valores
        x = np.sort(df[col].values)
        # Calculamos la ECDF
        y = np.arange(1, len(x) + 1) / len(x)
        
        # Graficamos la ECDF
        axes[i].plot(x, y, marker='.', linestyle='none', alpha=0.5)
        axes[i].plot(x, y, linestyle='-', linewidth=1)
        
        # Añadimos líneas para cuartiles
        for p in [0.25, 0.5, 0.75]:
            quantile = df[col].quantile(p)
            axes[i].axvline(x=quantile, color='r', linestyle='--', alpha=0.3)
            axes[i].text(quantile, 0.02, f'Q{int(p*100//25)}', ha='center', color='r')
        
        # Configuramos títulos y etiquetas
        axes[i].set_title(f'ECDF de {col}', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)
        axes[i].set_ylabel('Proporción acumulada', fontsize=12)
        axes[i].grid(True, alpha=0.3)
    
    # Ocultamos los ejes vacíos si el número de variables es impar
    if n_vars % 2 != 0:
        axes[-1].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Funciones de Distribución Acumulada Empírica (ECDF)', fontsize=18, y=1.005)
    return fig

In [ ]:
def plot_categorical_analysis(df, categorical_var, numerical_var=None):
    """
    Crea visualizaciones para variables categóricas con análisis opcional contra una variable numérica.
    
    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        categorical_var (str): Nombre de la variable categórica a analizar.
        numerical_var (str, optional): Nombre de la variable numérica para comparación. 
                                      Si no se proporciona, solo se muestra el análisis univariado.
    """
    fig, axes = plt.subplots(1, 3 if numerical_var else 1, figsize=(18, 6) if numerical_var else (8, 6))
    
    # Ajuste para cuando no hay variable numérica
    if not numerical_var:
        axes = [axes]  # Convertimos en lista para mantener consistencia en el acceso
    
    # Gráfico de conteo
    sns.countplot(x=categorical_var, data=df, ax=axes[0], palette='Set2')
    axes[0].set_title(f'Conteo por categorías de {categorical_var}', fontsize=14)
    axes[0].set_xlabel(categorical_var, fontsize=12)
    axes[0].set_ylabel('Conteo', fontsize=12)
    
    # Añadimos porcentajes
    total = len(df)
    for i, count in enumerate(df[categorical_var].value_counts().values):
        percentage = count / total * 100
        axes[0].annotate(f'{percentage:.1f}%', 
                         xy=(i, count), 
                         xytext=(0, 5),
                         textcoords='offset points',
                         ha='center')
    
    # Solo continuamos con los otros gráficos si hay variable numérica
    if numerical_var:
        # Boxplot de la variable numérica por categoría
        sns.boxplot(x=categorical_var, y=numerical_var, data=df, ax=axes[1], palette='Set2')
        axes[1].set_title(f'{numerical_var} por categorías de {categorical_var}', fontsize=14)
        axes[1].set_xlabel(categorical_var, fontsize=12)
        axes[1].set_ylabel(numerical_var, fontsize=12)
        
        # Violinplot de la variable numérica por categoría
        sns.violinplot(x=categorical_var, y=numerical_var, data=df, ax=axes[2], palette='Set2', inner='quartile')
        axes[2].set_title(f'Distribución de {numerical_var} por {categorical_var}', fontsize=14)
        axes[2].set_xlabel(categorical_var, fontsize=12)
        axes[2].set_ylabel(numerical_var, fontsize=12)
    
    plt.tight_layout()
    plt.suptitle(f'Análisis de la variable {categorical_var}', fontsize=18, y=1.05)
    return fig